In [16]:
import psycopg2
import requests
import json
import pandas as pd
import time
import urllib.request
import geopandas as gpd
import rasterio
import unrasterize
import datetime
import numpy as np
import os

In [ ]:
MAAS_ENDPOINT = "https://model-service.worldmodelers.com/"
#conn = psycopg2.connect(host="maas.cowfye1azh5f.us-east-1.rds.amazonaws.com",database="output", user="wmuser", password="REPLACEME")
#cur = conn.cursor()
user = "wmuser"
password = "Bq248mQHV9EDuUap"

In [ ]:
# get all training data (configs + outputs)
def get_training_data(model_name):
    response = requests.get(MAAS_ENDPOINT + "list_runs/" + model_name)
    run_ids = json.loads(response.text)
    
    datas = []
    configs = []
    for rid in run_ids:
        #rid = rid.lower()
        config_response = requests.get(MAAS_ENDPOINT + "run_results/" + rid)
        config = json.loads(config_response.text)        
        statement = f"SELECT * FROM rundata where run_id = '{rid}';"
        cur.execute(statement)
        print(cur.fetchall())
        output_df = pd.DataFrame(cur.fetchall(),columns=['run_id', 'x', 'y','id','feature','datetime'])
        datas.append(output_df)
        configs.append(config)
            
    return configs, datas

In [ ]:
np.arange(.0000000011,1.25,.24999999999)

In [ ]:
"""
{
   "config":{
      "_type": 'mm_data',
      "dekad": '01',
      "year": 2019
   },
   "name":"CHIRPS"
}
"""

import pandas as pd
import numpy as np
import itertools
d = pd.read_csv("/Users/jgawrilow/single_shock_results.csv")
countries = list(d.country)
#countries = ["ETH"]
_type = ["mm_data"]
dekad = map(lambda x: str(x).zfill(2),range(1,37))
year = list(range(2019,1980,-1))
mmatrix = list(itertools.product(*[_type,year,dekad]))
print(mmatrix)

In [ ]:
import itertools
run_matrix = {
    "x":range(0,100),
    "y":["A","B","C"],
    "z":["Z","Y","X"]
}

test = [[1,2,3],[4,5,6]]

result = list(itertools.product(*test))
print(result)

In [ ]:
"""
if params are 
-- x float [0-100]
-- y A B C
-- z Z Y Z
"""

run_matrix = {
    "x":range(0,100),
    "y":["A","B","C"],
    "z":["Z","Y","X"]
}

def range_to_list(x):
    vals = []
    k = x.keys()[0]
    for v in x[k]:
        vals.append({k:v})
        
#result = list(itertools.product(*inputdata))
    

# continuously run model and cache outputs
def run_and_cache(session,model_name,runs,fields):
    
    for j, run in enumerate(runs):
        mydata = {
           "config":{},
           "name":model_name
        }
        for i, field in enumerate(fields):
            mydata["config"][field] = run[i]

        URL = MAAS_ENDPOINT + "run_model"
        #print(json.dumps(mydata))
        resp = session.post(URL, json=mydata)
        rid = resp.text.replace('"',"").strip()
        time.sleep(2)
        run_response = session.get(MAAS_ENDPOINT + "run_status/" + rid)
        STATUS = run_response.text.replace('"',"").strip()
        print(j, rid, STATUS)

        while STATUS != "SUCCESS" and STATUS != "FAIL":
            time.sleep(1)
            URL = MAAS_ENDPOINT + "run_status/" + rid
            run_response = session.get(MAAS_ENDPOINT + "run_status/" + rid)
            STATUS = run_response.text.replace('"',"").strip()
            print(j, rid, STATUS)
        
        
        
    

In [ ]:
configs, datas = get_training_data("DSSAT")
print(datas)

In [ ]:
session = requests.Session()
session.auth = (user, password)
run_and_cache(session,"CHIRPS",mmatrix,["_type","year","dekad"])

In [ ]:
session = requests.Session()
session.auth = (user, password)
print(session.get(MAAS_ENDPOINT + "available_results").text)

In [ ]:
import wget
import zipfile
import os

def prep_p(session,model_name,runs,fields):
    
    dfs = {}
    out = open("chirps_runs.json","w")
    for run in runs:
        mydata = {
           "config":{},
           "name":model_name
        }
        for i, field in enumerate(fields):
            mydata["config"][field] = run[i]

        URL = MAAS_ENDPOINT + "run_model"
        print(json.dumps(mydata))
        resp = session.post(URL, json=mydata)
        rid = resp.text.replace('"',"").strip()
        dfs[rid] = mydata
        run_response = session.get(MAAS_ENDPOINT + "run_status/" + rid)
        STATUS = run_response.text.replace('"',"").strip()
    out.write(json.dumps(dfs,indent=2))
    out.close()
#         if STATUS == "SUCCESS":
#             URL = MAAS_ENDPOINT + "run_results"
#             run_response = session.get(MAAS_ENDPOINT + "run_results/" + rid)
#             outloc = json.loads(run_response.text)["output"]
#             wget.download(outloc,"./runs/")
            
            

In [ ]:
session = requests.Session()
session.auth = (user, password)
prep_p(session,"CHIRPS",mmatrix,["_type","year","dekad"])

# CHIRPS 2017 - 2019
Below pulls in anomaly and data information for 2017 - 2019.

These rasters are converted to coarse grained CSVs and prepped for Tableau ingestion.

## Run CHIRPS for 2017 - 2019

In [ ]:
def pad_dekad(d):
    if d < 10:
        return f"0{d}"
    else:
        return f"{d}"

In [ ]:
years = [2017, 2018, 2019]
dekads = [pad_dekad(i) for i in range(1, 36)]

In [ ]:
session = requests.Session()
session.auth = (user, password)

In [ ]:
URL = MAAS_ENDPOINT + "run_model"

x is longitude, y is latitude

In [ ]:
SW=(1.149689, 21.702228)
NE=(16.162925, 50.055588)
xmin = 21.702228
xmax = 50.055588
ymin = 1.149689
ymax = 16.162925

In [ ]:
for year in years:
    for dekad in dekads:
        payload = {
               "config":{
                  "_type": "mm_data",
                  "dekad": dekad,
                  "year": year,
                  "bbox": [xmin, ymin, xmax, ymax]
               },
               "name":"CHIRPS"
            }
        resp = session.post(URL, json=payload)

In [ ]:
URL = MAAS_ENDPOINT + "available_results"
CHIRPS_results = session.get(URL + "?ModelName=CHIRPS")
CHIRPS_results = [i for i in CHIRPS_results.json() if i['config']['config']['year']>=2017]
CHIRPS_results = [i for i in CHIRPS_results if i['config']['config']['_type']=='mm_data']
CHIRPS_results = [i for i in CHIRPS_results if 'bbox' in i['config']['config']]

In [ ]:
len(CHIRPS_results)

In [ ]:
rows = []
for i in CHIRPS_results:
    year = i['config']['config']['year']
    dekad = i['config']['config']['dekad']
    uri = i['output']
    obj = {"year": year, "dekad": dekad, "uri":uri}
    rows.append(obj)
    
df = pd.DataFrame(rows)
df = df.sort_values(['year','dekad']).reset_index()
df = df[['year','dekad','uri']]

In [ ]:
for kk, vv in df.iterrows():
    print(vv.year, vv.dekad)
    urllib.request.urlretrieve(vv.uri, f'CHIRPS/{vv.year}-{vv.dekad}.tiff')

## Run CHIRPS Anomalies for 2017 - 2019

In [ ]:
session = requests.Session()
session.auth = (user, password)

In [ ]:
URL = MAAS_ENDPOINT + "run_model"

x is longitude, y is latitude

In [ ]:
SW=(1.149689, 21.702228)
NE=(16.162925, 50.055588)
xmin = 21.702228
xmax = 50.055588
ymin = 1.149689
ymax = 16.162925

In [ ]:
for year in years:
    for dekad in dekads:
        payload = {
               "config":{
                  "_type": "mm_anomaly",
                  "dekad": dekad,
                  "year": year,
                  "bbox": [xmin, ymin, xmax, ymax]
               },
               "name":"CHIRPS"
            }
        resp = session.post(URL, json=payload)

In [ ]:
URL = MAAS_ENDPOINT + "available_results"
CHIRPS_results = session.get(URL + "?ModelName=CHIRPS")
CHIRPS_results = [i for i in CHIRPS_results.json() if i['config']['config']['year']>=2017]
CHIRPS_anomalies = [i for i in CHIRPS_results if i['config']['config']['_type']=='mm_anomaly']

In [ ]:
len(CHIRPS_anomalies)

In [ ]:
rows = []
for i in CHIRPS_anomalies:
    year = i['config']['config']['year']
    dekad = i['config']['config']['dekad']
    uri = i['output']
    obj = {"year": year, "dekad": dekad, "uri":uri}
    rows.append(obj)
    
df_anom = pd.DataFrame(rows)
df_anom = df_anom.sort_values(['year','dekad']).reset_index()
df_anom = df_anom[['year','dekad','uri']]

In [ ]:
for kk, vv in df_anom.iterrows():
    print(vv.year, vv.dekad)
    urllib.request.urlretrieve(vv.uri, f'CHIRPS-Anomalies/{vv.year}-{vv.dekad}.tiff')

## Convert CHIRPS to CSV

In [17]:
def gen_csv_from_CHIRPS(raster_path, out_path, mask_width=6, threshold=0):
    """
    Decrease mask_width for higher fidelity (finer grained); increase it for lower fidelity (course grained).
    """
    raster_data = rasterio.open(raster_path)
    
    # Since unrasterize *sums* across pixels as it simplifies
    # We need to count total pixels above our threshold to get the mean for the area of interest
    total_pixels = len(raster_data.read(1)[raster_data.read(1)>threshold])
    
    # threshold to zero means that no negative pixels (missing data) are included
    unrasterizer = unrasterize.WindowedUnrasterizer(mask_width=mask_width, threshold=threshold)
    unrasterizer.select_representative_pixels(raster_data)
    gdf = unrasterizer.to_geopandas(value_attribute_name='rainfall', crs=raster_data.crs)
    gdf['latitude'] = gdf.geometry.apply(lambda x: x.bounds[1])
    gdf['longitude'] = gdf.geometry.apply(lambda x: x.bounds[0])
    
    # Adjust rainfall by number of pixels (original) per new pixel
    # Basically, get the mean of pixels that were aggregated
    gdf['rainfall'] = gdf.rainfall.apply(lambda x: x/(total_pixels/gdf.shape[0]))
    
    gdf[['rainfall','latitude','longitude']].to_csv(out_path)
    print(f"Converted raster to {gdf.shape[0]} points")

In [18]:
chirps_mm = [i for i in os.listdir('CHIRPS') if 'tiff' in i]
chirps_anomalies = [i for i in os.listdir('CHIRPS-Anomalies') if 'tiff' in i]
chirps_mm.sort()
chirps_anomalies.sort()

In [19]:
for path in chirps_mm:
    in_path = f"CHIRPS/{path}"
    out_path = f"CHIRPS/{path.split('.tiff')[0]+'.csv'}"
    try:
        gen_csv_from_CHIRPS(in_path, out_path)
    except Exception as e:
        # likely due to a bad file for dates we have not reached yet        
        print(e)
        
for path in chirps_anomalies:
    in_path = f"CHIRPS-Anomalies/{path}"
    out_path = f"CHIRPS-Anomalies/{path.split('.tiff')[0]+'.csv'}"
    try:
        gen_csv_from_CHIRPS(in_path, out_path, threshold=-8000) # set a threshold, -9999 is missing        
    except Exception as e:
        # likely due to a bad file for dates we have not reached yet
        print(e)

Converted raster to 1332 points
Converted raster to 1230 points
Converted raster to 1360 points
Converted raster to 1359 points
Converted raster to 1482 points
Converted raster to 1568 points
Converted raster to 1862 points
Converted raster to 1857 points
Converted raster to 2058 points
Converted raster to 2171 points
Converted raster to 2250 points
Converted raster to 2417 points
Converted raster to 2558 points
Converted raster to 2582 points
Converted raster to 2574 points
Converted raster to 2467 points
Converted raster to 2192 points
Converted raster to 2327 points
Converted raster to 2493 points
Converted raster to 2434 points
Converted raster to 2455 points
Converted raster to 2454 points
Converted raster to 2481 points
Converted raster to 2549 points
Converted raster to 2437 points
Converted raster to 2403 points
Converted raster to 2524 points
Converted raster to 2538 points
Converted raster to 2448 points
Converted raster to 2379 points
Converted raster to 2208 points
Converte

### Process CHIRPS CSVs

In [20]:
chirps_mm = [i for i in os.listdir('CHIRPS') if 'csv' in i]
chirps_anomalies = [i for i in os.listdir('CHIRPS-Anomalies') if 'csv' in i]

In [21]:
first = True
for path in chirps_mm:
    df_ = pd.read_csv(f"CHIRPS/{path}")[['rainfall','latitude','longitude']]
    df_['year'] = path.split('-')[0]
    df_['dekad'] = path.split('-')[1].split('.csv')[0]
    df_['type'] = 'mm_data'
    if first == True:
        df_combined = df_
        first = False
    else:
        df_combined = df_combined.append(df_)
        
for path in chirps_anomalies:
    df_ = pd.read_csv(f"CHIRPS-Anomalies/{path}")[['rainfall','latitude','longitude']]
    df_['year'] = path.split('-')[0]
    df_['dekad'] = path.split('-')[1].split('.csv')[0]
    df_['type'] = 'mm_anomaly'
    df_combined = df_combined.append(df_)        

In [22]:
df_combined.shape

(466735, 6)

In [23]:
df_combined.groupby(["type","year"])['dekad'].count()

type        year
mm_anomaly  2017    94814
            2018    94527
            2019    69758
mm_data     2017    74613
            2018    75241
            2019    57782
Name: dekad, dtype: int64

In [24]:
def process_dekad(year, dekad):
    dt = datetime.datetime(int(year), 1, 1)
    dtdelta = datetime.timedelta(int(dekad) * 10 - 1)
    date = dt + dtdelta
    return date.strftime("%m-%d-%Y")

Convert dekad/year to a mm-dd-yyyy timestamp

In [25]:
df_combined['date'] = df_combined.apply(lambda x: process_dekad(x.year, x.dekad), axis=1)

In [26]:
df_combined["month"] = df_combined.date.apply(lambda x: int(x.split('-')[0]))

In [27]:
df_combined.head()

,rainfall,latitude,longitude,year,dekad,type,date,month
0,68.930994,4.570738,21.876941,2018,06,mm_data,03-01-2018,3
1,89.801743,7.081148,36.502882,2018,06,mm_data,03-01-2018,3
2,63.467025,7.130372,36.802389,2018,06,mm_data,03-01-2018,3
3,59.678886,6.195121,35.554442,2018,06,mm_data,03-01-2018,3
4,71.614373,6.096673,36.652635,2018,06,mm_data,03-01-2018,3


In [28]:
df_combined.to_csv('CHIRPS-combined.csv', index=False)